# 一步步搭建自己的循环神经网络

由于独特的记忆("memory")功能，循环神经网络(Recurrent Neural Network)在自然语言处理（NLP, Natual Language Processing）和其他处理序列的任务中有长足的应用。其可以在某个时间读取输入$x^{<t>}$（比如说是好多词），通过隐含层中从一个时间步到另一个时间步传递的激活值可以记录一些信息或者语义。比如单向的RNN（uni-directional RNN）可以获取来自过去的信息来产生后来的输出。而双向的RNN(bidirection RNN)可以从过去以及未来来获取语义信息。

**符号表示**：
- 方括号 $[l]$ 表明该对象和第$l$ 层相关
    - 例如：$a^{[4]}$表示第$4$层的激活（activation）。$W^{[5]}$和$b^{[5]}$是第$5$层的参数
- 圆括号 $(i)$ 表明该对象和第$i$个样本相关
    - 例如：$x^{(i)}$是第$i$个训练输入样本（example，周志华《🍉书》翻译成样例）
- 尖括号 $<t>$ 表明该对象在第$t$个事件步(time-step)
    - 例如：$x^{<t>}$是第$t$个时间步上的输入；$x^{(i)<t>}$是第$t$个时间步上的第$i$个样本
- 下标 $i$ 表明向量的第$i$个条目（entry）
    - 例如：$a_{i}^{[l]}$表明第$l$层的激活向量的第$i$个条目


首先来导入所需要的包

In [1]:
import numpy as np
from rnn_utils import *

## 1 - 基本的RNN的前向传播(Forward propagation)

基本的RNN结构如下，示例中，$T_{x}=T_{y}$。
<!-- ![图1: 基本RNN模型](images/RNN.png) -->

<img src= "images/RNN.png">
<caption><center>**图1. 基本RNN结构模型** </center></caption>

开始着手实现一个RNN：

**步骤**：
1. 实现RNN在单个时间步上的计算
2. 实现在$T_{x}$时间步上的循环，来一次处理所有的输入。
就是这个思路，下面开干！

### 1.1 - RNN 单元
一个RNN可以看作是一个单一的RNN单元的重复。下图描述的是一个RNN单元在单个时间步上的计算。
<img src="images/rnn_step_forward.png">
<caption><center>**图2. 基本RNN单元** 获取输入$x^{<t>}$（当前输入）和$a^{<t-1>}$（从过去信息中得到的前一个隐藏状态(hidden state)），输出$a^{<t>}$，接着传递给下一个RNN单元，并以之来预测得到$y^{<t>}$。</center></caption>

**实战**：实现图2中描述的RNN基本单元

**指令**：
1. 计算出tanh函数作为激活函数的隐藏值(hidden state):$a^{<t>}=tanh(W_{aa}a^{<t-1>}+W_{ax}x^{<t>}+b_{a})$
2. 使用新得到的隐藏值(hidden state)$a^{<t>}$，计算和预测$\hat{y}^{<t>}=softmax(W_{ya}a^{<t>}+b_y)$
3. 将计算值$(a^{<t>}, a^{<t-1>, x^{t}, parameters})$存储到元组cache中以便之后使用和返回
4. 返回$a^{<t>},y^{<t>},cache$

采用 $m$ 个样本，因此，$x^{<t>}$维度大小为$(n_{x}, m)$，$a^{<t>}$维度大小为$(n_{a}, m)$

In [4]:
# 实现函数：rnn_cell_forward

def rnn_cell_forward(xt, a_prev, parameters):
    """
    对于单个RNN单元，实现如图2所示的单个前向步骤
    
    参数：
    xt -- 在 t 时间步上的输入数据，numpy数组，大小为(n_x, m)
    a_prep -- t-1 时间步的hidden state, numpy数组，大小为(n_a, m)
    parameters -- python 字典，包含：
                        Wax -- 和输入相乘的权重矩阵，numpy数组，大小为(n_a, n_x)
                        Waa -- 和隐藏状态相称的权重矩阵，numpy数组，大小为(n_a, n_a)
                        Wya -- 隐藏-输出之间相关的权重矩阵，numpy数组，大小为(n_y, n_a)
                        ba -- 偏差，numpy数组，大小是(n_a, 1)
                        by -- 偏差，隐藏-输出之间相关，numpy数组，大小为(n_y, 1)
    返回：
    a_next -- 下一个隐藏状态(hidden state)， 大小为(n_a, m)
    yt_pred -- 在时间步"t"上的预测值，numpy数组，大小为(n_y, m)
    cache -- 反向传播所需要的变量组成的元组，包含(a_next, a_prep, xt, parameters)
                        
    """
    
    # 从"parameters" 中回取参数
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    # 根据上方给定的公式来计算下一个激活状态
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    # 根据上方给定的公职来计算当前单元的输出
    yt_pred = softmax(np.dot(Wya, a_next)+ by)
    
    # 存储反向传播所需要的中间变量
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache
    
    

In [5]:
np.random.seed(1)
xt = np.random.randn(3,10)
a_prev = np.random.randn(5,10)
Waa = np.random.randn(5,5)
Wax = np.random.randn(5,3)
Wya = np.random.randn(2,5)
ba = np.random.randn(5,1)
by = np.random.randn(2,1)
parameters = {"Waa": Waa, "Wax": Wax, "Wya": Wya, "ba": ba, "by": by}

a_next, yt_pred, cache = rnn_cell_forward(xt, a_prev, parameters)
print("a_next[4] = ", a_next[4])
print("a_next.shape = ", a_next.shape)
print("yt_pred[1] =", yt_pred[1])
print("yt_pred.shape = ", yt_pred.shape)

a_next[4] =  [ 0.59584544  0.18141802  0.61311866  0.99808218  0.85016201  0.99980978
 -0.18887155  0.99815551  0.6531151   0.82872037]
a_next.shape =  (5, 10)
yt_pred[1] = [ 0.9888161   0.01682021  0.21140899  0.36817467  0.98988387  0.88945212
  0.36920224  0.9966312   0.9982559   0.17746526]
yt_pred.shape =  (2, 10)


### 1.2 - RNN 前向过程

可以将RNN是做是刚才实现的单元的重复，加入你的输入序列包含10个时间步，那么久需要将刚才的RNN单元重复十次。每个单元接收来自前一个单元的隐藏状态($a^{\langle t-1\rangle}$)和当前时间步上的输入数据($x^{\langle t\rangle}$),其输出一个隐藏状态($a^{<t>}$)以及当前时间步上的预测($y^{\langle t\rangle}$)。

<img src="images/rnn1.png">
<caption>
<center>**图3. 基本RNN。**输入序列 $x=(x^{\langle 1\rangle}, x^{\langle 2 \rangle},...,x^{\langle T_x\rangle})$包含$T_x$个时间步。网络的输出是$y=(y^{\langle 1 \rangle}, y^{\langle 2\rangle},...,y^{\langle T_{x} \rangle})$
</center>
</caption>

**实战**：实现图3中的RNN的前向传播过程。

**指令**：
1. 创建一个全零向量 ($a$) 用来存储RNN中计算过程中得到的全部隐藏状态
2. 初始化"next"隐藏状态为$a_0$（即初始的隐藏状态）
3. 开始在每个时间步上循环，递增的索引值是$t$：
    - 运行`rnn_cell_forward`函数对"下一个"隐藏状态和cache进行更行
    - 将"下一个"隐藏状态的值存入$a$（第$t$的位置）
    - 将预测值存储到变量y中
    - 将cache值加入到列表caches中
4. 返回$a$,$y$和caches



In [ ]:
# 实现函数：rnn_forward

def rnn_forward(x, a0, parameters):
    """
    实现RNN的前向传播过程
    
    参数：
    x -- 每个时间步上的输入数据，大小为(n_x, m, T_x)
    a0 -- 初始的隐藏状态，大小为(n_a, m)
    parameters -- python 字典包含：
                        Waa -- 和隐藏状态相乘的权重矩阵，numpy数组，大小为(n_a, n_a)
                        Wax -- 和输入状态相乘的权重矩阵，numpy数组，大小为(n_a, n_x)
                        Wya -- 隐藏-输出之间相关的权重矩阵，numpy数组，大小为(n_y, n_a)
                        ba -- 偏差，numpy 数组，大小为(n_a, 1)
                        by -- 隐藏-输出相关的偏差，numpy数组，大小为(n_y, 1)
    返回：
    a -- 每个时间步上的隐藏状态，numpy数组，大小为(n_a, m, T_x)
    y_pred -- 对于每个时间步骤得到的预测，numpy数组，大小为(n_y, m, T_x)
    cache -- 用于反向传播的值，包含(caches列表, x)
    """
    # 初始化caches 列表，其中包含所有的cache
    caches = []
    
    # 从x的大小和parameters["Wya"]回取维数
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
    
    # 使用全零矩阵来初始化 "a" 和 "y"
    a = None
    y_pred = None
    
    # 初始化 a_next 
    a_next = None
    
    # 在所有时间步上遍历
    for t in range(None):
        # 更新下一个隐藏状态，计算预测值，得到cache 
        a_next, yt_pred, cache = None
        # 将新的
    